# Download SRA annotations based on GEO accessions

## Load libraries and define analysis-specific variables

In [19]:
suppressPackageStartupMessages(library(GEOquery))
suppressPackageStartupMessages(library(reutils))
suppressPackageStartupMessages(library(xml2))
suppressPackageStartupMessages(library(glue))
suppressPackageStartupMessages(library(tidyverse))

## Load GEO datasets that we want to download annotations and download GEO record

In [22]:
geo_accessions <- read_csv("../annotations/geo_accession_numbers.csv",
  comment = "#",
  show_col_types = F
) %>%
  mutate(geo = map(gse, ~getGEO(.x, GSEMatrix = FALSE))) %>%
  print()

Using locally cached version of GSE145723 found here:
/tmp/RtmpM56ACw/GSE145723.soft.gz 

Reading file....

Parsing....

Found 11 entities...

GPL20301 (1 of 12 entities)

GSM3907597 (2 of 12 entities)

GSM3907598 (3 of 12 entities)

GSM3907599 (4 of 12 entities)

GSM3907600 (5 of 12 entities)

GSM4176097 (6 of 12 entities)

GSM4176098 (7 of 12 entities)

GSM4331576 (8 of 12 entities)

GSM4331577 (9 of 12 entities)

GSM4331578 (10 of 12 entities)

GSM4331579 (11 of 12 entities)



# A tibble: 1 × 21
  study  study_subset gse     pmid organism rrna_index trim  adapter trim5 trim3
  <chr>  <chr>        <chr>  <dbl> <chr>    <chr>      <chr> <lgl>   <lgl> <lgl>
1 han20… monosome     GSE1… 3.24e7 human    hg38.rrna  no    NA      NA    NA   
# … with 11 more variables: trim_condition <lgl>, comment <lgl>,
#   transcript_index <chr>, transcript_annotations <chr>, sample_subset <lgl>,
#   codon_annotations <chr>, align_parameters <chr>,
#   vk_type_motif_annotations <chr>, hisat2_index <chr>,
#   strand_stall_motif_annotations <chr>, geo <list>


## Define functions for parsing specific meta-data values

In [5]:

get_bioproject_number <- function(geo) {
  Meta(geo) %>% 
  enframe("annotation", "value") %>%
  unnest() %>%
  mutate(bioproject_number = str_extract(value, "PRJNA[:digit:]+")) %>%
  filter(!is.na(bioproject_number)) %>%
  pull(bioproject_number)
}

get_gsm_meta <- function(gsm) {
  Meta(gsm) %>% 
  enframe("annotation", "value") %>%
  unnest()
}

get_srx_number <- function(gsm) {
  gsm %>% 
    mutate(srx = str_extract(value, "SRX[:digit:]+")) %>%
    filter(!is.na(srx)) %>%
    pull(srx)
}
    
get_gsm_title <- function(gsm) {
  gsm %>% 
    filter(annotation == "title") %>% 
    pull(value) %>% 
    janitor::make_clean_names()
}

get_gsm_organism <- function(gsm) {
  gsm %>% 
    filter(str_detect(annotation, "organism")) %>% 
    pull(value) %>% 
    janitor::make_clean_names()
}

get_gsm_celltype <- function(gsm) {
  gsm %>%
    mutate(celltype = str_extract(value, "293|((?<=cell line: |tissue: |cell type: ).+)")) %>%
    filter(!is.na(celltype)) %>%
    slice(1) %>% 
    pull(celltype) %>% 
    janitor::make_clean_names()
}

## Parse Experiment and Project annotations

In [24]:
geo_annotations <- geo_accessions %>%
  mutate(prjna = map_chr(geo, get_bioproject_number)) %>%
  mutate(gsm = map(geo, GSMList)) %>%
  select(-geo) %>%
  mutate(gsm_number = map(gsm, names)) %>%
  unnest(cols = c(gsm, gsm_number)) %>%
  mutate(gsm = map(gsm, get_gsm_meta)) %>%
  mutate(sample_title = map_chr(gsm, get_gsm_title)) %>%
  mutate(srx = map_chr(gsm, get_srx_number)) %>%
  mutate(organism = map_chr(gsm, get_gsm_organism)) %>%
  mutate(cell_type = map_chr(gsm, get_gsm_celltype)) %>%
  select(-gsm) %>%
  rename(gsm = gsm_number) %>%
  select(gsm, gse, prjna, srx, cell_type) %>%
  group_by(gsm) %>%
  slice(1) %>%
  ungroup() %>%
  mutate(cell_type = if_else(str_detect(cell_type, "embryonic_kidney"),
    "HEK293T", cell_type
  )) %>%
  print()

Warning message:
“`cols` is now required when using unnest().
Please use `cols = c(value)`”
Warning message:
“`cols` is now required when using unnest().
Please use `cols = c(value)`”
Warning message:
“`cols` is now required when using unnest().
Please use `cols = c(value)`”
Warning message:
“`cols` is now required when using unnest().
Please use `cols = c(value)`”
Warning message:
“`cols` is now required when using unnest().
Please use `cols = c(value)`”
Warning message:
“`cols` is now required when using unnest().
Please use `cols = c(value)`”
Warning message:
“`cols` is now required when using unnest().
Please use `cols = c(value)`”
Warning message:
“`cols` is now required when using unnest().
Please use `cols = c(value)`”
Warning message:
“`cols` is now required when using unnest().
Please use `cols = c(value)`”
Warning message:
“`cols` is now required when using unnest().
Please use `cols = c(value)`”
Warning message:
“`cols` is now required when using unnest().
Please use `cols =

# A tibble: 10 × 5
   gsm        gse       prjna       srx        cell_type
   <chr>      <chr>     <chr>       <chr>      <chr>    
 1 GSM3907597 GSE145723 PRJNA607993 SRX6369535 hek293   
 2 GSM3907598 GSE145723 PRJNA607993 SRX6369536 hek293   
 3 GSM3907599 GSE145723 PRJNA607993 SRX6369537 hek293   
 4 GSM3907600 GSE145723 PRJNA607993 SRX6369538 hek293   
 5 GSM4176097 GSE145723 PRJNA607993 SRX7182523 hek293   
 6 GSM4176098 GSE145723 PRJNA607993 SRX7182524 hek293   
 7 GSM4331576 GSE145723 PRJNA607993 SRX7778690 hek293   
 8 GSM4331577 GSE145723 PRJNA607993 SRX7778691 hek293   
 9 GSM4331578 GSE145723 PRJNA607993 SRX7778692 hek293   
10 GSM4331579 GSE145723 PRJNA607993 SRX7778693 hek293   


## Get SRA Run information for the BioProject number associated with each study

In [10]:
esearch_query <- geo_annotations %>% 
  distinct(prjna) %>% 
  pull() %>% 
  paste0(collapse = " or ") %>% 
  print()

esearch_result <- esearch(esearch_query, db = "sra")
efetch_result <- efetch(esearch_result, db = "sra")

[1] "PRJNA607993"


## Parse XML file to extract SRA information

In [25]:
sra_info <- read_xml(content(efetch_result, as = "text"))

sra_annotations <- sra_info %>% 
  xml_find_all("//EXPERIMENT") %>% 
  map_df(xml_attrs) %>%
  rename(srx = accession, gsm = alias) %>% 
  mutate(sample_name = map(gsm, function (x) xml_text(xml_find_all(sra_info, glue('//SAMPLE[@alias="{x}"]/TITLE'))))) %>%
  mutate(data = map(srx, function (x) bind_rows(xml_attrs(xml_find_all(sra_info, glue('//EXPERIMENT_REF[@accession="{x}"]/..')))))) %>%
  unnest(c(sample_name, data)) %>%
  select(gsm, srx, sample_name, accession, total_spots, total_bases) %>%
  type_convert() %>%
  rename(srr = accession) %>%
  mutate(sample_name = janitor::make_clean_names(sample_name)) %>%
  select(srr, everything()) %>%
  left_join(select(geo_annotations, gsm, gse, cell_type), by = "gsm") %>%
  select(-srx, -total_spots, -total_bases) %>%
  left_join(select(geo_accessions, study_subset, gse), by = "gse") %>%
  filter(str_detect(sample_name, tolower(study_subset)) | is.na(study_subset)) %>%
  select(-study_subset) %>%
  write_tsv("../annotations/sra_annotations.tsv") 


── Column specification ────────────────────────────────────────────────────────
cols(
  gsm = col_character(),
  srx = col_character(),
  sample_name = col_character(),
  accession = col_character(),
  total_spots = col_double(),
  total_bases = col_double()
)



## Print SRA annotations

In [18]:
sra_annotations

srr,gsm,sample_name,gse,cell_type
<chr>,<chr>,<chr>,<chr>,<chr>
SRR9604620,GSM3907598,monosome_rep2,GSE145723,hek293
SRR9604619,GSM3907597,monosome_rep1,GSE145723,hek293
